# SK Memory - RAG Pattern Primer

Learning objectives:

- Setting up SK to save memories
- Saving, retrieving, recalling, and deleting memories

Reference:
- https://github.com/Azure-Samples/azure-search-openai-demo
- https://github.com/azure-samples/semantic-kernel-rag-chat

**Important:**

- A memory includes at a minium a key, some text (a text chunk), and the embedding
- The embedding is used to perform a semantic search, but the text is what is used for augmentation

## Load the required .NET packages and supporting classes

In [ ]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.47.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.47.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.47.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.47.0-alpha"


using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;

using dotenv.net;

const string MemoryCollectionName = "aboutMe";

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var adaDeploymentName = "text-embedding-3-large";
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

## Get a Kernel with an embedding service

In [ ]:
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var ramStore = new Microsoft.SemanticKernel.Memory.VolatileMemoryStore();

var embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(
            deploymentName: adaDeploymentName,
            endpoint: endpoint,
            apiKey: apiKey);

var memory = new MemoryBuilder()
            .WithTextEmbeddingGeneration(embeddingGenerator)
            .WithMemoryStore(new Microsoft.SemanticKernel.Memory.VolatileMemoryStore())
            .Build();

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey)
    .Build();

## Save memories - Method 1

In [ ]:
// ========= Store memories using the kernel =========

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andre");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I work as a tourist operator");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I've been living in Seattle since 2005");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");

## Save memories - Method 2 - using a skill

In [ ]:
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var memoryPlugin = kernel.ImportPluginFromObject(new Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin(memory));

In [ ]:
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var result = await kernel.InvokeAsync(memoryPlugin["Save"], new()
        {
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.InputParam] = "My family is from New York",
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.KeyParam] = "info5",
        });

Console.WriteLine(result);

## Retrive a memory by Key

In [ ]:
// ========= Test memory remember =========
Console.WriteLine("========= Example: Recalling a Memory =========");

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var result = await kernel.InvokeAsync(memoryPlugin["retrieve"], new()
        {
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.KeyParam] = "info5",
        });
Console.WriteLine(result);

## Recall a memory based on relevance and a count limit

In [ ]:
Console.WriteLine("========= Example: Recalling an Idea =========");

 await foreach (var answer in memory.SearchAsync(
            collection: MemoryCollectionName,
            query: "where did I grow up?",
            limit: 2,
            minRelevanceScore: 0.3,
            withEmbeddings: true))
        {
            Console.WriteLine($"Answer: {answer.Metadata.Text}");
        }

## Recall a memory using an SK function

In [ ]:
Console.WriteLine("========= Example: Using Recall in a Semantic Function =========");

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var result = await kernel.InvokeAsync(memoryPlugin["Recall"], new()
        {
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.InputParam] = "Ask: my family is from?",
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.LimitParam] = "2",
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.RelevanceParam] = "0.5",
        });

 Console.WriteLine($"Answer: {result.GetValue<string>()}"); 

## Remove a memory by key

In [ ]:
Console.WriteLine("========= Example: Remove a memory by key =========");

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
var result = await kernel.InvokeAsync(memoryPlugin["Remove"], new()
        {
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.CollectionParam] = MemoryCollectionName,
            [Microsoft.SemanticKernel.Plugins.Memory.TextMemoryPlugin.KeyParam] = "info5"
        });
Console.WriteLine(result);

## Get a list of collections

In [ ]:
Console.WriteLine("========= Example: Get a list of collections =========");
var collections = await memory.GetCollectionsAsync();
foreach (var collection in collections)
{
    Console.WriteLine(collection);
}

## Remove a memory using the textMemory functionality

In [ ]:
Console.WriteLine("========= Example: Remove a memory by key =========");
await memory.RemoveAsync(MemoryCollectionName, "info1");
